In [2]:
import torch, transformers, json
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)

## Dataset value example 
{'record': 'JAHFAU010000168.1', 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACCCGGCGGCGCACCTCGAGCGATTCCTCCACCACGTCGTAGCCGGCGACTTTGGCGCTCCCGCTCGTTGGCGGCATGTATCCGGTGAGGATCCGCATCGTGGTCGTCTTCCCGGCGCCGTTCGGTCCAAGGAATCCCAGGATCTCGCCACGTTTGGCCTCGAAGGAAATTCCCTCGAGCGCGGTCAGGCTCCCGTACGTTTTCGAAAGCTTATTGGCCTCGATCAAGCCTCGGTCCTCAAGCCCAGCTCGCCAGGGGGATGGCCGGACACAAAAACATCCCGGCTCTGCCGTTGCGAAGATCTTGCTACGGAAGACCGGGATGTGAGGGATCCGATCCCGGTGACCACTATACAGAGATTCGCGGCCGGGAGTCAAGACGCAGTTGCGCTTCTAATGAGACGCTAAGCTCGCGGCCCGGTTTTCTTTAAGTTTGCTTGACATGGCATGTGCCAACTTGTACAATCGCCCAGCCTCCCCCACAGGAGCCTTAGAATCGGATGCAAGCTCTGACCGCATTTGAACAAAGCGCGGTTCTCGGCGTTGTCGGGATTGCCATACTGGGGTTGGTCTACGCACTTTTCCTGCGGACCCAGGTGCTGCGCGAACCGGCCGGGTCCCCCCGGATGCAGGAGGTCTGGAATGCGATCCGGGTAGGCGCCGACACTTACCTCCGCCGCCAGCTGGGCAGCATCCTCCCGCTGATCGCGGTCCTCACGGTCGCCCTCTTCCTGAGCGTCTACATCGTCCCCCCCAGCCCGGAGGCGACAGAACGATTCAAAACTTTGGATTCGGACACCGTCCGGCTGGTGATCGGGTTGGGAAGAGCCTTGGCCTTCGTCATGGGGGCCATCTTTAGCGTGACCGTTGGGCAGATTGGGATGCGGATGGCGGTGCAGGCCAATGTTCGAGTCGCCGCGGCGGCCCGCCGGTCGTTCGGGGACTCGCTGCGCATCGCCTACCGGGCCGGCACGATCACCGGGATGTTGACCGACGGACTCGGCCTTTTCGGCGGAACGATGATCTTCATCATCTTCGGAATCGCCTCGCCGGATGCGCTGCTCGGGTTCGGATTCGGCGGGACCCTGGTGGCGCTCTTCATGCGGGTTGGCGGGGGGATCTACACCAAGGCGGCCGACGTCGGGGCGGACCTGGTCGGCAAGATCGAGGCGGGTATCCCCGAGGACGACCCACGCAATCCCGCAGTTATCGCCGATCTGGTGGGCGACAACGTGGGCGACTGCGCCGGGATGGCGGCGGATATCTTTGAGTCATATGAAGTCACGATCGTCTCCGGTCTGATTCTTGGGCTGGCGCTTTGGCACATCACCGGCAACTTCGAATGGGTGGTCTACCCCCTGCTCGTGCGCGGCATCGGTGTGCTTTCTTCGATCATCGGCACCTACGCGGTCCGCGGCGGCCATGGGAAGAGCGGCAATGCGATGCGGGCGATCTTCACCGGGTTCTTGTCCTCGGCGGCGATCTCGGTGGTCTTGTTCGGAATCATGGGATTCTTTTACATGCGAGAGGTGCCGGGAGGCTGGTGGCGGCCGTTCCTGGCCACGACCGCGGGGGTGCTGCTGGCGATCGCCATCGACCGACTGACCGAGTACTTCACCGGAAGCCACGGCACTCCGGTTCAGGAAATCCGCAAGGCCACCAACACCGGCCCGGCGACCACCATTCTCTCCGGGATCGCGGTTGGGTATGAGTCGAGTGTCTGGTCGATCCTGATCATCGCGGTAACCATCTTCGTCTCGATCCTTACTTTCGGCGGAATCTCCGGCATCCCGGCCGGCGAGGCGGTGACCTTTGTGCTGTATGGCGTCGCCATGACCGGCATCGGCATGCTGAATCTGACCGGAAACAACGTGGCGATGGATTCCTACGGACTGATCGCCGACAATGCCAATGGAATCGGCCAAATGGCCTGGCAGGGGGAGGACGACCGCGATACGGTGCAGGCCCGCCAGATCATGGCCGACCTGGACGCGGTGGGAAACACGACCAAGGC'}

In [3]:
# let's try prompting the fine-tuned models
def continued_convo(model, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", model)
    if convo==None:
        messages=[{"role": "system", "content": """You are a helpful assistant, 
        if I ask you a question you can't answer, please reply 'I can't answer that' """}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=256)[0]['generated_text'][-1]
    print(answer)
    messages.append(answer)
    return pipe, messages 

In [5]:
mis_model=AutoModelForCausalLM.from_pretrained('LoRA_Training/fine_tuned_mis/')

Loading checkpoint shards: 100%|██████| 2/2 [00:02<00:00,  1.18s/it]


In [10]:
lla_model=AutoModelForCausalLM.from_pretrained('./fine_tuned_lla2/') 
# compatability issue, needs debugging

Loading checkpoint shards: 100%|██████| 2/2 [00:02<00:00,  1.11s/it]


RuntimeError: Error(s) in loading state_dict for MistralForCausalLM:
	size mismatch for model.embed_tokens.weight: copying a param with shape torch.Size([128256, 2048]) from checkpoint, the shape in current model is torch.Size([32000, 4096]).
	size mismatch for model.layers.0.self_attn.q_proj.lora_A.default.weight: copying a param with shape torch.Size([8, 2048]) from checkpoint, the shape in current model is torch.Size([8, 4096]).
	size mismatch for model.layers.0.self_attn.q_proj.lora_B.default.weight: copying a param with shape torch.Size([2048, 8]) from checkpoint, the shape in current model is torch.Size([4096, 8]).
	size mismatch for model.layers.0.self_attn.v_proj.lora_A.default.weight: copying a param with shape torch.Size([8, 2048]) from checkpoint, the shape in current model is torch.Size([8, 4096]).
	size mismatch for model.layers.0.self_attn.v_proj.lora_B.default.weight: copying a param with shape torch.Size([512, 8]) from checkpoint, the shape in current model is torch.Siz

In [ ]:
# both of the models above are the ones pretrained using steps as the eval_strategy